In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install Tensor Flow Model Maker
!pip install -q tflite-model-maker

     |████████████████████████████████| 642 kB 33.9 MB/s 
     |████████████████████████████████| 10.9 MB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 61.9 MB/s 
     |████████████████████████████████| 87 kB 7.0 MB/s 
     |████████████████████████████████| 1.2 MB 27.4 MB/s 
     |████████████████████████████████| 237 kB 77.4 MB/s 
     |████████████████████████████████| 596 kB 35.6 MB/s 
     |████████████████████████████████| 120 kB 80.0 MB/s 
     |████████████████████████████████| 3.4 MB 58.7 MB/s 
     |████████████████████████████████| 6.4 MB 62.2 MB/s 
     |████████████████████████████████| 840 kB 48.8 MB/s 
     |████████████████████████████████| 42.6 MB 1.7 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 48.3 MB 84 kB/s 
     |████████████████████████████████| 352 kB 68.4 MB/s 
     |████████████████████████████████| 99 kB 9.8 MB/s 
     |██████████████

In [ ]:
# Import Library
!sudo apt-get install libportaudio2
!pip install -q tflite-model-maker

import os
import sounddevice as sd
import numpy as np
import tensorflow as tf

assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

from PIL import Image as pil
import matplotlib.pyplot as plt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 0s (190 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype


In [ ]:
# Buat Folder Hasil
folderHasil = '/content/drive/MyDrive/Colab Notebooks/Hasil'
folderBatubara = '/content/drive/MyDrive/Colab Notebooks/Batubara/Foto'

for namaFolder in os.listdir(folderBatubara):
  f = os.path.join(folderHasil, namaFolder)
  if not os.path.exists(f):
    os.makedirs(f)

In [ ]:
# Pre-processing Foto
for namaFolder in os.listdir(folderBatubara):
  index = 1
  d = os.path.join(folderBatubara, namaFolder)
  for namaFoto in os.listdir(d):
    f = os.path.join(d, namaFoto)
    data = pil.open(f)

    # Flip Foto Kanan Ke Kiri
    data.transpose(pil.FLIP_LEFT_RIGHT).save(folderHasil+'/'+namaFolder+'/'+str(index)+'.JPG')
    index += 1
    
    # Rotate Foto 90 Derajat
    data.transpose(pil.ROTATE_90).save(folderHasil+'/'+namaFolder+'/'+str(index)+'.JPG')
    index += 1

    # Flip Foto Atas Ke Bawah
    data.transpose(pil.ROTATE_90).transpose(pil.FLIP_TOP_BOTTOM).save(folderHasil+'/'+namaFolder+'/'+str(index)+'.JPG')
    index += 1

In [2]:
import tensorflow as tf

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/Colab Notebooks/Hasil/',
    shuffle=True,
    image_size = (256,256),
    batch_size = 8
)

Found 957 files belonging to 3 classes.


In [3]:
class_names = dataset.class_names
class_names

['Rendah', 'Sedang', 'Tinggi']

In [4]:
data_train = dataset.take(int(0.8*len(dataset)))
data_val = dataset.skip(len(data_train)).take(int(0.1*len(dataset)))
data_test = dataset.skip(len(data_train)).skip(len(data_val))

In [5]:
# from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input

# base_model = VGG16(input_shape = (256,256,3),
#                    include_top = False,
#                    weights= 'imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [6]:
# for layer in base_model.layers:
#   layer.trainable = False

In [7]:
# vgg_preprocess = preprocess_input
# data_train.preprocessing_function = vgg_preprocess

In [1]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

# model = Sequential()
# model.add(base_model)

# model.add(Flatten())

# model.add(Dense(units=128, kernel_initializer='uniform', activation='relu'))
# # model.add(Dense(units=32, kernel_initializer='uniform', activation='relu'))
# model.add(Dense(units=3, kernel_initializer='uniform', activation = 'softmax'))

# model.compile(optimizer = 'adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])

NameError: ignored

In [54]:
!pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 2.9 MB/s 


In [63]:
from keras.models import Model
from keras.layers import Flatten,Activation,Input,Conv2D,MaxPooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.applications.imagenet_utils import obtain_input_shape

input_shape=(256,256,3)
input_shape = obtain_input_shape(input_shape,
                                  default_size=256,
                                  min_size=48,
                                  data_format=K.image_data_format(),
                                  require_flatten=True,
                                  weights='imagenet')

img_input = Input(shape=input_shape)

# Block 1   
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

x = GlobalMaxPooling2D()(x)

model = Model(img_input, x, name='vgg16')

In [65]:
from keras.utils.data_utils import get_file

WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                        WEIGHTS_PATH_NO_TOP,
                        cache_subdir='models')

In [67]:
model.load_weights(weights_path)

In [72]:
from tensorflow.keras.preprocessing.image import img_to_array

x_train=[]
y_train=[]

for images,labels in data_train.take(96):
  n=0
  for image in images:
    img = img_to_array(images[n]/255)
    n+=1
    img=np.expand_dims(img,axis=0)
    img_encode = model.predict(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
  for label in labels:
    lbl = label.numpy()
    y_train.append([lbl])

In [73]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [83]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

def create_model(optimizer='adam',init_mode='uniform',activation='relu',neurons=128):
  model = Sequential()

  model.add(Flatten())

  model.add(Dense(units=neurons, kernel_initializer=init_mode, activation=activation))
  model.add(Dense(units=3, kernel_initializer=init_mode, activation = 'softmax'))

  model.compile(optimizer = optimizer,loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])
  return model

In [84]:
from keras.wrappers.scikit_learn import KerasClassifier

tuning_model = KerasClassifier(build_fn=create_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
len(data_train)

96

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],batch_size = [8,16,32],epochs=[5,10,15],init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],activation=['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],neurons = [16,32,64,128,256])
grid = GridSearchCV(estimator = tuning_model,param_grid=param_grid,cv=3) 
grid_result = grid.fit(x_train,y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


64/64 [==============================] - 0s 2ms/step - loss: 1.0988 - accuracy: 0.3379
Epoch 2/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0983 - accuracy: 0.3477
Epoch 3/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0980 - accuracy: 0.3418
Epoch 4/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0978 - accuracy: 0.3438
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 1.0973 - accuracy: 0.3555
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


64/64 [==============================] - 1s 2ms/step - loss: 1.0988 - accuracy: 0.2949
Epoch 2/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0987 - accuracy: 0.3320
Epoch 3/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0984 - accuracy: 0.3301
Epoch 4/5
64/64 [==============================] - 0s 2ms/step - loss: 1.0983 - accuracy: 0.3477
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 1.0971 - accuracy: 0.3516
Epoch 1/5


...

64/64 [==============================] - 1s 3ms/step - loss: 1.0855 - accuracy: 0.3867
Epoch 2/5
64/64 [==============================] - 0s 3ms/step - loss: 1.0366 - accuracy: 0.4863
Epoch 3/5
64/64 [==============================] - 0s 3ms/step - loss: 1.0069 - accuracy: 0.5176
Epoch 4/5
64/64 [==============================] - 0s 3ms/step - loss: 0.9815 - accuracy: 0.5781
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.9616 - accuracy: 0.5781
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


64/64 [==============================] - 1s 3ms/step - loss: 1.0906 - accuracy: 0.3887
Epoch 2/5
64/64 [==============================] - 0s 3ms/step - loss: 1.0470 - accuracy: 0.5078
Epoch 3/5
64/64 [==============================] - 0s 3ms/step - loss: 1.0186 - accuracy: 0.5488
Epoch 4/5
64/64 [==============================] - 0s 3ms/step - loss: 0.9915 - accuracy: 0.5684
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.9871 - accuracy: 0.4727
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


19/64 [=======>......................] - ETA: 0s - loss: 1.0987 - accuracy: 0.3684 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4194432   
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 18,913,347
Trainable params: 4,198,659
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.fit(data_train,batch_size=8, epochs=5, validation_data = data_val)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


96/96 [==============================] - 97s 799ms/step - loss: 0.3958 - accuracy: 0.8385 - val_loss: 0.0709 - val_accuracy: 0.9792
Epoch 2/5
96/96 [==============================] - 41s 417ms/step - loss: 0.0248 - accuracy: 0.9922 - val_loss: 0.0695 - val_accuracy: 0.9792
Epoch 3/5
96/96 [==============================] - 41s 418ms/step - loss: 0.0107 - accuracy: 0.9961 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 4/5
96/96 [==============================] - 41s 417ms/step - loss: 4.3032e-04 - accuracy: 1.0000 - val_loss: 0.0179 - val_accuracy: 0.9896
Epoch 5/5
96/96 [==============================] - 41s 416ms/step - loss: 2.1238e-04 - accuracy: 1.0000 - val_loss: 0.0120 - val_accuracy: 0.9896


In [ ]:
model.evaluate(data_test)

12/12 [==============================] - 24s 253ms/step - loss: 0.0350 - accuracy: 0.9785


[0.035041529685258865, 0.9784946441650391]

In [ ]:
model.save('my_model.h5') 
model = tf.keras.models.load_model('my_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpjp9hdbc6/assets


75662852